<span style="color: Gold"> 개체명 인식: NER
- 텍스트에서 특정 의미를 가진 단어나 구절을 찾아내고 분류하는 작업
- “여기서 사람 이름은 뭐야?”, “여기서 장소는 어디야?”, “조직 이름이 있니?”, “시간/날짜/수량 같은 정보는?” 이런 내용을 자동으로 찾아주는 NLP 기술
- 자연어처리의 기초적인 테스크

<span style="font-size:12px;">텍스트</span>


- <예시문> 홍길동은 2025년 11월 19월 서울시청에서 삼성전자 직원을 만났다
  - 홍길동 - [인명]
  - 2024년 1월 19일 - [날짜]
  - 서울시청 - [지명]
  - 삼성전자 - [기관명]

- 활용분야
  - 뉴스기사 : 기사에서 인물, 장소, 기관 자동추출
  - 의료문서 : 병명, 약물명, 증상
  - 계약서 : 회사명, 날짜, 금액
  - 쳇봇 : 사용자 질문에 핵심정보 파악

- BIO 태깅
  - B(Being) 개체 시작
  - I(inside) 개체 내부
  -  O (Outside) 개체가 아님

<span style="font-size:12px;">

### <span style="color: lightblue;"> BERT 기반 NER 모델 개발 과정 요약

| 단계 (Stage) | 주요 목표 및 역할 | 핵심 코드/개념 | 문맥적 의미 (비유) |
| :---: | :--- | :--- | :--- |
| **1. 데이터 준비** | 훈련할 **'교과서'**를 만들고 컴퓨터 언어로 번역합니다. | `train_labels`, `tokenizer(...)`, `is_split_into_words=True` | 원본 텍스트를 **숫자(텐서)**로 변환하고 정답 라벨(`PER`, `LOC` 등)을 준비합니다. |
| **2. 모델 정의 및 테스트** | **AI 뇌(설계도)**를 만들고 데이터 흐름을 1차 점검합니다. | `SimpleNERModel`, `torch.argmax(dim=-1)`, `id2label` | **BERT**를 기반으로 분류층을 연결한 후, 예측값(`logits`)을 뽑아내는 과정이 잘 작동하는지 확인합니다. |
| **3. 모델 학습 (Training)** | 모델에게 반복적으로 문제를 풀게 하여 **지식(가중치)**을 쌓게 합니다. | `loss.backward()`, `optimizer.step()`, `zero_grad()` | **Loss(오차)**를 채점하고, **역전파**를 통해 오차를 수정하여 모델이 점차 똑똑해지게 만듭니다. |
| **4. 모델 평가 및 활용** | 학습된 모델의 **실력을 측정**하고 실전에 투입합니다. | Accuracy, F1-Score | 학습하지

---

<span style="color: Gold"> 1. 데이터 준비
- 1. 라벨(Tag) 정의 및 인코딩
    - 사용할 모든 개체명 태그(PER, LOC, ORG, O 등)를 정의
    - 각 태그를 고유한 숫자 ID로 매핑하는 딕셔너리(label2id, id2label)를 생성
- 2. 토크나이징
    - tokenizer를 사용하여 문장을 토큰 ID로 변환
    - BERT 특수 토큰([CLS], [SEP])을 추가하고, 최대 길이에 맞춰 자르기(Truncation) 및 패딩(Padding) 처리를 수행
- 3. 라벨 정렬
    - 토크나이징으로 인해 쪼개진 단어(Subword)에 맞춰 정답 라벨도 복사하거나 마스킹(Masking)하여 길이를 맞춘다
    - 패딩된 토큰의 라벨은 손실 계산 시 무시되도록 -100 등의 값으로 설정
- 4. 데이터 로드 생성
    - 가공된 데이터를 담는 Dataset 객체를 만든다
    - 데이터를 배치(Batch) 단위로 묶어 모델에게 공급할 DataLoader를 생성

In [25]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [26]:
# BIO 태깅
tokens = ["김철수는", "2024년", "1월", "15일", "서울시청에서", "삼성전자", "직원을", "만났다"]
bio_tags = ["B-PER", "B-DAT", "I-DAT", "I-DAT", "B-LOC", "B-ORG", "O", "O"] # B-DAT 처음 나온 날짜의 정보, I-DAT 앞에 날짜의 정보가 있으면 b가 아닌 i로 
for token, tag in zip(tokens, bio_tags):
    if tag.startswith('B-'):
        desc = f"'{tag[2:]}' 개체의 시작"
    elif tag.startswith('I-'):
        desc = f"'{tag[2:]}' 개체의 내부"
    else :
        desc = "개체가 아님"
    print(f" {token:12} | {tag:8} | {desc}")

 김철수는         | B-PER    | 'PER' 개체의 시작
 2024년        | B-DAT    | 'DAT' 개체의 시작
 1월           | I-DAT    | 'DAT' 개체의 내부
 15일          | I-DAT    | 'DAT' 개체의 내부
 서울시청에서       | B-LOC    | 'LOC' 개체의 시작
 삼성전자         | B-ORG    | 'ORG' 개체의 시작
 직원을          | O        | 개체가 아님
 만났다          | O        | 개체가 아님


In [27]:
# 학습데이터

train_sentences = [
    ["김철수는", "서울에", "산다"],
    ["이영희는", "2024년에", "부산으로", "이사했다"],
    ["삼성전자는", "대한민국의", "대기업이다"],
    ["박지성은", "축구선수다"],
    ["2025년", "1월", "1일은", "새해다"],
]

train_labels = [
    ["B-PER", "B-LOC", "O"],
    ["B-PER", "B-DAT", "B-LOC", "O"],
    ["B-ORG", "B-LOC", "O"],
    ["B-PER", "O"],
    ["B-DAT", "I-DAT", "I-DAT", "O"],
]


In [28]:
%pip install sentencepiec

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sentencepiec (from versions: none)
ERROR: No matching distribution found for sentencepiec


In [29]:
from ast import mod
# 토크나이져
MODEL_NAME = 'skt/kobert-base-v1'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
text = '김철수는 서울에 산다'
#토크나이져
tokens = tokenizer.tokenize(text)
# 인코딩
encoded = tokenizer(text,return_tensors='pt')
encoded

{'input_ids': tensor([[517, 490, 494,   0, 517,   0, 491,   0, 491,   0, 517,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

---

<span style="color: Gold"> 2. 모델 정의 및 순전파 테스트

- 1. BERT 모델 로드
    - 사전 학습된 **BERT 모델(AutoModel)**을 로드
- 2. koBERT 인코더: 문장의 의미를 이해
- 3. 분류기(Liner): 예측
- 4. 출력 라벨:  B-PER 0 B-LOC

In [31]:

import torch.nn as nn
import numpy as np
class SimpleNERModel(nn.Module):
  def __init__(self, num_labels) -> None:
    super(SimpleNERModel, self).__init__()
    self.num_labels = num_labels
    self.bert = AutoModel.from_pretrained(MODEL_NAME)
    self.dropout = nn.Dropout(0.1)
    self.clf = nn.Linear(self.bert.config.hidden_size,  self.num_labels)
  def forward(self, input_ids, attention_mask):
    # kobert로 문자 인코딩
    outputs = self.bert(input_ids, attention_mask=attention_mask)
    # 마지막 은닉상태 추출
    sequence_output =  outputs.last_hidden_state
    # Dropout 적용
    sequence_output = self.dropout(sequence_output)
    # 분류기
    logits = self.clf(sequence_output)
    return logits
# 라벨의 개수
label_list = sorted(list(set([data for i in train_labels for data in i])))
label2id = { label:i for i, label in enumerate(label_list)}
id2label = { i:label for i, label in enumerate(label_list)}

model = SimpleNERModel(num_labels=len(label_list))

# 모델 학습
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# 순전파 테스트
# 1. 문장과 정답 꺼내기
sample_sentence = train_sentences[0]
sample_label = train_labels[0]
print(f"테스트문장 {''.join(sample_sentence)}")
print(f"정답라벨 {''.join(sample_label)}")
# 2. 토크나이저로 숫자 변환
encoding = tokenizer(sample_sentence,
                     return_tensors='pt',   
                     truncation = True,     
                     padding = True,        
                     max_length = 32, 
                     is_split_into_words = True)  # 이미 띄어쓰기 된 리스트야
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

with torch.no_grad(): # 지금은 학습 아니니까 기록하지마
  model.eval()        # 평가모드이기 떄문에 dropout 끄기
  logits = model(input_ids, attention_mask)   # 모델 통과 (결과:logits)
  predictions = torch.argmax(logits, dim=-1)  # 가장 높은 점수 찍기(argmax), 각 단어마다 7개 라벨 후보 중 1등을 뽑아라
                                              # 7개인걸 어떻게 알 수 있느냐?->label_list = set([data for i in train_labels for data in i])# 예: label_list = {'PER', 'LOC', 'ORG', 'DATE', 'TIME', 'ETC', 'O'} -> 총 7개!
# 숫자를 다시 글자로
word_ids =  encoding.word_ids(batch_index=0)
pred_labels = []

for i, word_idx in enumerate(word_ids):
  if word_idx is not None and i < len(predictions[0]):  # 특수 토큰(CLS, SEP)이나 패딩은 건너뛰어줘
    pred_label = id2label[ predictions[0][i].item() ]   # 모델이 예측한 번호를 실제 라벨 이름으로 바꾸기 (예: 1 -> 'PER')
    if word_idx < len(sample_sentence):
      print(f'{sample_sentence[word_idx]:10} -> {pred_label:8} 정답 : {sample_label[word_idx]}')


테스트문장 김철수는서울에산다
정답라벨 B-PERB-LOCO
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
김철수는       -> O        정답 : B-PER
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> B-ORG    정답 : B-LOC
서울에        -> O        정답 : B-LOC
서울에        -> O        정답 : B-LOC
산다         -> O        정답 : O
산다         -> O        정답 : O


In [82]:
# 학습 DataSet
from torch.utils.data import Dataset
class NerDataSet(Dataset):
  def __init__(self,sentences,labels, tokenizer,max_len=64) -> None:
    self.sentences = sentences
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.sentences)
  def __getitem__(self,idx):
    words = self.sentences[idx]
    lbls = self.labels[idx]
    encoding = self.tokenizer(
        words,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=self.max_len,
        is_split_into_words=True
    )
    word_ids = encoding.word_ids(batch_index = 0)
    label_ids = []
    for w in word_ids:
      if w is None:
        label_ids.append(-100)
      else:
        label_ids.append(label2id[lbls[w]])
    return {
        'input_ids' : encoding['input_ids'].squeeze(),
        'attention_mask' : encoding['attention_mask'].squeeze(),
        'labels' : torch.tensor(label_ids)
    }

In [83]:
# DataLoader
from torch.utils.data import DataLoader
train_dataset = NerDataSet(train_sentences,train_labels,tokenizer,max_len=10)
train_loader = DataLoader(train_dataset,batch_size=2,shuffle=True)

---

<span style="color: Gold"> 3. 모델 학습

In [84]:
# 모델 선언 및 학습
criterion = nn.CrossEntropyLoss(ignore_index=-100)
model = SimpleNERModel(num_labels=len(label_list))
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [85]:
for epoch in range(10):
  model.train()
  total_loss = 0
  for batch in train_loader:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    logits = model(input_ids, attention_mask)
    loss = criterion(logits.view(-1, model.num_labels), labels.view(-1))
    total_loss += loss.item()
    loss.backward()
    optimizer.step()
  print(f'epoch {epoch+1} loss : {total_loss/len(train_loader):.4f}')


epoch 1 loss : 1.8437
epoch 2 loss : 1.5022
epoch 3 loss : 1.3574
epoch 4 loss : 1.3915
epoch 5 loss : 1.4190
epoch 6 loss : 1.5183
epoch 7 loss : 1.2911
epoch 8 loss : 1.3610
epoch 9 loss : 1.2155
epoch 10 loss : 1.1997


In [87]:
sample_sentence, sample_label =  train_sentences[0], train_labels[0]
print(sample_sentence, sample_label)
encoding = tokenizer(
        sample_sentence,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=20,
        is_split_into_words=True
    )
print(encoding)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)
model.eval()
with torch.no_grad():
  logits = model(input_ids, attention_mask)
  predictions = torch.argmax(logits, dim=-1)[0]
print('결과')
word_ids = encoding.word_ids(batch_index=0)
for i ,word_idx in enumerate(word_ids):
  if word_idx is not None:
    pred_label = id2label[predictions[i].item()]
    print(f'{sample_sentence[word_idx]} -> {pred_label} 정답 : {sample_label[word_idx]}')
     

['김철수는', '서울에', '산다'] ['B-PER', 'B-LOC', 'O']
{'input_ids': tensor([[517, 490, 494,   0, 517,   0, 491,   0, 491,   0, 517,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
결과
김철수는 -> B-PER 정답 : B-PER
김철수는 -> B-PER 정답 : B-PER
김철수는 -> B-PER 정답 : B-PER
김철수는 -> B-PER 정답 : B-PER
서울에 -> B-PER 정답 : B-LOC
서울에 -> B-PER 정답 : B-LOC
서울에 -> B-PER 정답 : B-LOC
서울에 -> B-PER 정답 : B-LOC
서울에 -> B-PER 정답 : B-LOC
서울에 -> B-PER 정답 : B-LOC
산다 -> B-PER 정답 : O
산다 -> B-PER 정답 : O


---

전체과정 흐름
